In [ ]:
import pickle
from glob import glob

In [ ]:
coverage_runs = glob('../dadvi_coverage_results/*/*.pkl')
model_names = [x.split('/')[-2] for x in coverage_runs]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'filename': coverage_runs, 'model_name': model_names})

In [ ]:
example = df.iloc[6]

example

In [ ]:
from scipy.stats import norm

In [ ]:
import numpy as np

def compute_z_scores(loaded):

    # Compute the z-scores
    z_scores = (loaded['means'] - loaded['reference_means']) / loaded['reference_freq_sds']

    # Make them an array
    z_array = np.stack(z_scores.values)
    
    return z_array

def evaluate_z_scores(z_array, crit_prob=0.025):

    z_crit = norm.ppf(crit_prob)
    within_interval = np.abs(z_array) < np.abs(z_crit)
    within_ratio = within_interval.mean()
    
    return within_ratio

In [ ]:
df['loaded'] = df['filename'].apply(pd.read_pickle)

In [ ]:
df['z_scores'] = df['loaded'].apply(compute_z_scores)

In [ ]:
df['frac_within'] = df['z_scores'].apply(evaluate_z_scores)

In [ ]:
df['M'] = df['loaded'].apply(lambda x: x['M'].iloc[0])

In [ ]:
df[['model_name', 'M', 'frac_within']].sort_values('frac_within')

In [ ]:
import matplotlib.pyplot as plt

df['frac_within'].hist()